In [52]:
from src.datamodules.components.nv_fetcher import NVDatasetFetcher
import pandas as pd
import numpy
import plotly.express as px
from matplotlib import pyplot as plt

In [ ]:
def lag_plot_single_vessel(vessel_vascu_activity: pd.Series):
    # Lag Plotting:
    pd.plotting.lag_plot(vessel_vascu_activity)
    plt.show()

    # Print correlation:
    values = pd.DataFrame(vessel_vascu_activity)
    dataframe = pd.concat([values.shift(1), values], axis=1)
    dataframe.columns = ['t', 't+1']
    result = dataframe.corr()
    print("(Auto) Correlation between the activity in `t` and `t + 1`")
    print(result)

    # Prints all the auto correlation
    from statsmodels.graphics.tsaplots import plot_acf
    plot_acf(vessel_vascu_activity, lags=31)

In [ ]:
# Load fetcher
fetcher = NVDatasetFetcher(data_dir=r'C:\Users\MatanBT\My Drive\University\Year 3\Semester B\ML-Workshop\neurovascular-project\data')
df_vascu = fetcher.get_vessels_df()
df_neuro = fetcher.get_neurons_df()

df_vascu

In [ ]:
# -------------- Auto-Correlations --------------
# Plot auto correlation of a SINGLE vessel
# (Demonstrates the strong auto-correlation of vascular activity)
INSPECTED_VESSEL_NAME = 'vessel_0'
lag_plot_single_vessel(df_vascu[INSPECTED_VESSEL_NAME])

In [67]:
# -------------- Correlations --------------
ves = df_vascu['vessel_0']
neu = df_neuro['neuron_0']

def two_timeseries_correlation(series_1=ves, series_2=neu, mode="corr", corr_window_len=1):
    assert mode in ['corr', 'corr_window', 'cross_corr']
    if mode == 'corr' or (mode == 'corr_window' and corr_window_len == 1):
        # Calculates simple pairwise correlation between the two time series.
        corr_value = series_1.corr(series_2)
        
    elif mode == 'corr_window':
        # Calculates pairwise correlation of series-1 and window-averages (lenghed `corr_window_len`) of series-2 (only).
        series_2 = series_2.rolling(corr_window_len, 
                                    center=False)  # True for centering the windows
        series_2 = series_2.mean()  # take mean on each window
        series_1, series_2 = series_1.iloc[corr_window_len:], series_2.iloc[corr_window_len:]  # cut NANs from window
        corr_value = series_1.corr(series_2)
        
    elif mode == 'cross_corr':
        # Calculates cross-correlation of the two time series.
        pass
    
    # print (f">> R={corr_value}")
    return corr_value

# two_timeseries_correlation()  # Sanity check
    

In [75]:
# Calculate on all correlations
n_count, v_count = df_neuro.shape[1], df_vascu.shape[1]
corr_array = np.zeros((n_count, v_count))
CORR_MODE, CORR_WIND_LEN = 'corr_window', 5
for n in range(n_count):
    for v in range(v_count):
        curr_n, curr_v = df_neuro.iloc[:, n], df_vascu.iloc[:, v]
        corr_array[n, v] = two_timeseries_correlation(curr_v, curr_n, mode=CORR_MODE, corr_window_len=CORR_WIND_LEN)

corr_array = np.abs(corr_array)  # we care for the absolute correlation

In [76]:
# Table summary
from scipy import stats
print(f"Corr Summary (mode={CORR_MODE}, window_len={CORR_WIND_LEN}): ")
pd.DataFrame(corr_array.flatten()).describe()

Corr Summary (mode=corr_window, window_len=5): 


,0
count,21300.000000
mean,0.038666
std,0.032104
min,0.000006
25%,0.014748
50%,0.031080
75%,0.054116
max,0.285616


In [ ]:
# Visual summary
fig = px.imshow(corr_array)
fig.update_layout(
        title=f"Corr Summary (mode={CORR_MODE}, window_len={CORR_WIND_LEN}): ",
        yaxis_title='Neurons',
        xaxis_title='Blood-Vessels',
    )
fig.write_html("./corr_summary_heatmap.html")
fig.show()

In [ ]:
# Additional plots
fig = px.scatter(corr_array.max(axis=0))
# Might indicate on a single blood-vessel that has strong correlation
fig.update_layout(
        title=f"Max correlation of each Blood Vessel (mode={CORR_MODE}, window_len={CORR_WIND_LEN}) (Higher is better) ",
        yaxis_title='Correlation',
        xaxis_title='#Blood-Vessel',
    )
fig.write_html("./max_corr_blood_vessels.html")
fig.show()

In [ ]:
print("means:", corr_array.mean(axis=0).mean())
print("maxs:" , corr_array.max(axis=0).mean())

In [96]:
# Additional plots
fig = px.scatter(corr_array.max(axis=0))
# Might indicate on a single blood-vessel that has strong correlation
fig.update_layout(
        title=f"Max correlation of each Blood Vessel (mode={CORR_MODE}, window_len={CORR_WIND_LEN}) (Higher is better) ",
        yaxis_title='Correlation',
        xaxis_title='#Blood-Vessel',
    )
fig.write_html("./max_corr_blood_vessels.html")
fig.show()

In [93]:
print("means:", corr_array.mean(axis=0).mean())
print("maxs:" , corr_array.max(axis=0).mean())

means: 0.03866642026283466
maxs: 0.12981049785857768


In [ ]:
"""
# Draft for AutoRegression
#----------------------------------------
# VARMAX example
import numpy as np
def to_positive_definitive(M):
    M = np.matrix(M)
    M = (M + M.T) * 0.5
    k = 1
    I = np.eye(M.shape[0])
    w, v = np.linalg.eig(M)
    min_eig = v.min()
    M += (-min_eig * k * k + np.spacing(min_eig)) * I
    return M

from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.metrics import mean_squared_error
# fit model
# model = VARMAX(df_vascu[['vessel_0', 'vessel_0']].values[:2000, :], exog=df_neuro[['neuron_0']].values[:2000, :], order=(2,0))
# model_fit = model.fit(disp=False)
# # make prediction
# yhat = model_fit.forecast(exog=df_neuro[['neuron_0']].values[2000:, :])
# print(yhat)

from statsmodels.tsa.ar_model import AutoReg
ar_model = AutoReg(df_vascu['vessel_0'], lags=8).fit()
# print(ar_model.summary())
pred = ar_model.forecast(901).tolist()
#
# Plot the prediction vs test data
error = mean_squared_error(df_vascu['vessel_0'].values[2000:2901], pred)
print(error)
"""